In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%pip install itables
#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

# Web scrapping de metacritic.com

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output


# Initialisation du DataFrame avec les colonnes requises
df = None

headers = {"User-Agent": "Mozilla/5.0"}  # Indiquer au site Metacritic que ce n'est pas un script qui accède au site mais bien un navigateur
base_url = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page="

# Initialisation du compteur de lignes
row_counter = 0

for i in range(0, 203):  # Itération sur les 204 pages du site
    critic_review_mixed = "N/A"
    critic_review_negative = "N/A"
    user_review_positive = "N/A"
    user_review_negative = "N/A"
    user_review_mixed = "N/A"
    
    print("Page en cours ", i )
    response = requests.get(base_url + str(i), headers=headers)  # URL de base + str(i) pour passer à la page suivante
    soup = BeautifulSoup(response.content, 'html.parser')
    game_links = soup.find_all("a", {"class": "title"})  # Trouver tous les liens de jeux sur la page
    
    for link in game_links:
        game_url = "https://www.metacritic.com" + link.get('href')  # Créer l'URL du jeu en ajoutant l'URL spécifique du jeu à l'URL de base
        game_response = requests.get(game_url, headers=headers)
        game_soup = BeautifulSoup(game_response.content, 'html.parser')
        print(game_url)
        
        game_name_section = game_soup.find("h1")
        game_name = game_name_section.text.strip() if game_name_section else "N/A"  # Extraire le nom du jeu du h1, ou renvoyer "N/A" si aucun h1 n'est trouvé

        score_section = game_soup.find('a', {'class': 'metascore_anchor'})
        if score_section:
            div = score_section.find('div')
            score = div.text if div else "N/A"  # Extraire le score du jeu de la balise div, ou renvoyer "N/A" si aucune balise div n'est trouvée
        else:
            score = "N/A"
            
        Nbr_Awards_section = game_soup.find("div" , {"class" : "body"})
        if Nbr_Awards_section:
            Nbr_Awards = len(Nbr_Awards_section.find_all("a"))
        else:
            Nbr_Awards = "N/A"
        
        value = game_soup.find("div", {"class": "details side_details"})
        if value:
            vote_count_section = value.find('span', {'class': 'count'})
            vote_count = vote_count_section.find("a").text if vote_count_section and vote_count_section.find("a") else "N/A"
        else:
            vote_count = "N/A"
            
        
        Nbr_Critics_section = game_soup.find("div", {"class": "summary"})
        if Nbr_Critics_section:
            Nbr_Critics_section = Nbr_Critics_section.find("a")
            Nbr_Critics_section = Nbr_Critics_section.find('span')
            Nbr_Critics = Nbr_Critics_section.find("a").text if Nbr_Critics_section and Nbr_Critics_section.find("a") else "N/A"
        else:
            Nbr_Critics = "N/A"

        user_score_section = game_soup.find("div", {"class": "userscore_wrap feature_userscore"})
        user_score = user_score_section.find('a').text.strip() if user_score_section and user_score_section.find('a') else "N/A"  # Extraire le score de l'utilisateur du jeu de la balise 'a', ou renvoyer "N/A" si aucune balise 'a' n'est trouvée
        
        years_section = game_soup.find("li", {"class" : "summary_detail release_data"})
        if years_section is not None:
            years_section = years_section.find("span", {"class" : "data"})
            if years_section:
                release_date = years_section.text
                year = years_section.text[-4:].strip()
            else:
                release_date = "N/A"
                year = "N/A"
        else:
            release_date = "N/A"
            year = "N/A"
        
        
        try : 
            critic_review_section = game_soup.find("ol", {"class" : "score_counts hover_none"})
            if critic_review_section is not None:
                critic_review_list = critic_review_section.findAll("li")
                critic_review_positive = critic_review_list[0].find("span" , {"class" : "count"}).text
                critic_review_mixed = critic_review_list[1].find("span" , {"class" : "count"}).text
                critic_review_negative = critic_review_list[2].find("span" , {"class" : "count"}).text
            
            else:
                critic_review_positive = "N/A"
                critic_review_mixed = "N/A"
                critic_review_negative= "N/A"
                
        except IndexError:
            critic_review_positive = "N/A"
            critic_review_mixed = "N/A"
            critic_review_negative= "N/A"

    
        try:
            user_review_list = game_soup.findAll("ol", {"class" : "score_counts hover_none"})[1].findAll("li")
            user_review_positive = user_review_list[0].find("span" , {"class" : "count"}).text
            user_review_mixed = user_review_list[1].find("span" , {"class" : "count"}).text
            user_review_negative = user_review_list[2].find("span" , {"class" : "count"}).text
        
        except IndexError:
            user_review_positive = "N/A"
            user_review_mixed = "N/A"
            user_review_negative = "N/A"

        
        platform = game_url.split("/")[4]  # Récupérer le nom de la plateforme du jeu à partir de l'URL du jeu

        # Création de la nouvelle ligne à ajouter au DataFrame
        new_row = {"Name": game_name, "Platform": platform, "Score": score, "Nbr_User_Critics": vote_count,
                   "User_score": user_score, "Year": year, "Nbr_Awards" : Nbr_Awards, "Release Date" : release_date,
                   "Critic_review_Positive" : critic_review_positive, "critic_review_mixed" : critic_review_mixed,
                   "critic_review_negative" : critic_review_negative, "user_review_positive" : user_review_positive,
                   "user_review_negative" : user_review_negative, "user_review_mixed" : user_review_mixed, "game_url" : game_url}
        
        
        
        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

        # Mise à jour du compteur de lignes
        row_counter += 1

        # Affichage du compteur de lignes toutes les 7 lignes
        if row_counter % 7 == 0:
            print(f"Processed {row_counter} rows.")
            # Pause pour permettre l'affichage du compteur
            # Clear print statements after they're printed
            clear_output(wait=True)

        # Pause de 0,05 seconde pour éviter un éventuel blocage par le site
        time.sleep(0.05)

https://www.metacritic.com/game/pc/postal-iii
https://www.metacritic.com/game/wii-u/game-party-champions
https://www.metacritic.com/game/game-boy-advance/legends-of-wrestling-ii
https://www.metacritic.com/game/xbox/pulse-racer
https://www.metacritic.com/game/xbox-one/fighter-within
https://www.metacritic.com/game/pc/flatout-3-chaos-destruction
https://www.metacritic.com/game/ds/homie-rollerz


In [ ]:
df = final_version_metacritic
# On souhaite faire correspondre les plateformes sur metacritic aux dénominations présentes sur VGChartz, pour leur future fusion :
dictionnaire = {"nintendo-64" : "N64",
               "playstation" : "PS",
               "playstation-3" : "PS3",
               "dreamcast" : "DC",
               "xbox-360" : "X360",
               "wii" : "Wii",
               "xbox-one" : "XOne",
               "pc" : "PC",
               "switch" : "SWITCH",
               "playstation-2" : "PS2",
               "playstation-4" : "PS4",
               "gamecube" : "GC",
               "xbox" : "XB",
               "wii-u" : "WiiU",
               "xbox-series-x" : "XBS",
               "playstation-5" : "PS5",
               "game-boy-advance" : "GBA",
               "3ds" : "3DS",
               "ds" : "DS",
               "playstation-vita" : "PSV",
               "psp" : "PSP",
               "stadia" : "STD"}



final_version_metacritic["Platform"] = final_version_metacritic["Platform"].replace(dictionnaire)

# Résultats

On récupère grace au webscrapping des jeux sur metacritic des variables intéressantes et utiles à notre future analyse des ventes, on dispose ainsi à présent des éléments suivants :
- Score moyen obtenu auprès de la presse spécialisée
- Score moyen obtenu auprès des joueurs
- Nombre de critiques rédigées par la presse
- Nombre de critiques rédigées par les joueurs
- Nombre de critiques par appréciation (positive, négative ou mixte/mitigé) pour les joueurs et pour la presse

In [4]:
pd.read_pickle('final_version_metacritic.pkl')

Name Platform Score Nbr_User_Critics  \
0       The Legend of Zelda: Ocarina of Time      N64    99             7922   
1                   Tony Hawk's Pro Skater 2       PS    98              910   
2                        Grand Theft Auto IV      PS3    98             4975   
3                                SoulCalibur       DC    98              416   
4                        Grand Theft Auto IV     X360    98             3729   
...                                      ...      ...   ...              ...   
20286                                 Gnosia   SWITCH    82       50 Ratings   
20287               MotorStorm: Pacific Rift      PS3    82      375 Ratings   
20288             Time Crisis: Project Titan       PS    71       10 Ratings   
20289                        Boulder Dash-XL     X360    71              N/A   
20290  Star Wars Galaxies: An Empire Divided       PC    71      106 Ratings   

      User_score  Year  Nbr_Awards  Release Date Critic_review_Positive  \
0            9.1  1998        16.0  Nov 23, 1998                     22   
1            7.5  2000        20.0  Sep 20, 2000                     19   
2            7.9  2008         4.0  Apr 29, 2008                     64   
3            8.5  1999        16.0  Sep  8, 1999                     24   
4            8.0  2008         4.0  Apr 29, 2008                     86   
...          ...   ...         ...           ...                    ...   
20286        8.4  2021         4.0  Mar  4, 2021                     34   
20287        8.0  2008         4.0  Oct 28, 2008                     54   
20288        6.5  2001        21.0  Jun 20, 2001                      4   
20289        tbd  2011         4.0  Jul 13, 2011                     12   
20290        5.8  2003         4.0  Jul  9, 2003                     15   

      critic_review_mixed critic_review_negative user_review_positive  \
0                       0                      0                1,638   
1                       0                      0                   76   
2                       0                      0                  631   
3                       0                      0                   77   
4                       0                      0                  701   
...                   ...                    ...                  ...   
20286                   4                      1                   15   
20287                   9                      0                   46   
20288                   9                      0                    3   
20289                   7                      2                    0   
20290                  14                      4                   22   

      user_review_negative user_review_mixed  \
0                       95                91   
1                       19                13   
2                      112               163   
3                        2                 4   
4                      113               185   
...                    ...               ...   
20286                    1                 3   
20287                    3                 6   
20288                    1                 1   
20289                    0                 1   
20290                   21                 4   

                                                game_url  \
0      https://www.metacritic.com/game/nintendo-64/th...   
1      https://www.metacritic.com/game/playstation/to...   
2      https://www.metacritic.com/game/playstation-3/...   
3      https://www.metacritic.com/game/dreamcast/soul...   
4      https://www.metacritic.com/game/xbox-360/grand...   
...                                                  ...   
20286      https://www.metacritic.com/game/switch/gnosia   
20287  https://www.metacritic.com/game/playstation-3/...   
20288  https://www.metacritic.com/game/playstation/ti...   
20289  https://www.metacritic.com/game/xbox-360/bould...   
20290  https://www.metacritic.com/game/pc/star-wars-g...   

   